In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Temp_hack/data/Test/test_data.csv")
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Mahindra XUV500 W6 2WD,Kochi,2017,41290,Diesel,Manual,Second,16.0 kmpl,2179 CC,140 bhp,7.0,NaN
1,BMW 5 Series 520d Luxury Line,Delhi,2013,65000,Diesel,Automatic,First,22.48 kmpl,1995 CC,190 bhp,5.0,67.87 Lakh
2,Toyota Corolla Altis 1.8 G,Bangalore,2016,16471,Petrol,Manual,Second,14.28 kmpl,1798 CC,138.03 bhp,5.0,20.48 Lakh
3,Hyundai i20 Asta 1.4 CRDi,Pune,2015,54339,Diesel,Manual,Second,22.54 kmpl,1396 CC,88.73 bhp,5.0,NaN
4,Mahindra KUV 100 mFALCON G80 K2,Hyderabad,2016,24025,Petrol,Manual,First,18.15 kmpl,1198 CC,82 bhp,6.0,NaN


In [6]:
df.drop('New_Price', axis=1, inplace=True)

In [10]:
df['Engine'].fillna('0 CC', inplace=True)
df['Power'].fillna('0 bhp', inplace=True)
df['Seats'].fillna(df['Seats'].mean(), inplace=True)
df['Mileage'].fillna(df['Mileage'].mean(), inplace=True)

TypeError: ignored

In [12]:


df['Mileage'] = df['Mileage'].str.extract('(\d+)').astype(float)
df['Engine'] = df['Engine'].str.extract('(\d+)').astype(int)
df['Power'] = df['Power'].str.extract('(\d+)').astype(float)


In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Location']=encoder.fit_transform(df['Location'])
df['Fuel_Type']=encoder.fit_transform(df['Fuel_Type'])
df['Transmission']=encoder.fit_transform(df['Transmission'])
df['Owner_Type']=encoder.fit_transform(df['Owner_Type'])
# df['Price']=encoder.fit_transform(df['Price'])

In [15]:
mean_engine = int(df['Engine'].mean())
mean_power = int(df['Power'].mean())

df['Engine'].replace(0, mean_engine, inplace=True)
df['Power'].replace(0, mean_power, inplace=True)

In [16]:
df['Power'].fillna(df['Power'].mean(), inplace=True)

In [17]:
df.rename(columns = {'Year':'Car_age'}, inplace = True)
df['Car_age'] = 2023 - df['Car_age']

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df[['Location','Car_age','Kilometers_Driven','Mileage','Engine','Power','Seats']] = sc.fit_transform(df[['Location','Car_age','Kilometers_Driven','Mileage','Engine','Power','Seats']])

In [19]:
X=df.drop(['Name'],axis=1)

In [20]:
X['Fuel_Type']=X['Fuel_Type'].astype(float)
X['Transmission']=X['Transmission'].astype(float)
X['Owner_Type']=X['Owner_Type'].astype(float)

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           2520 non-null   int64  
 1   Car_age            2520 non-null   int64  
 2   Kilometers_Driven  2520 non-null   int64  
 3   Fuel_Type          2520 non-null   int64  
 4   Transmission       2520 non-null   int64  
 5   Owner_Type         2520 non-null   int64  
 6   Mileage            2518 non-null   float64
 7   Engine             2520 non-null   float64
 8   Power              2520 non-null   float64
 9   Seats              2520 non-null   float64
dtypes: float64(4), int64(6)
memory usage: 197.0 KB


In [23]:
X.fillna(X.Mileage.mean(), inplace=True)

In [24]:
for i in X.columns:
    X[i]=X[i].astype(int)

In [25]:
import pickle
file = open('/content/drive/MyDrive/Temp_hack/my_ensemble.pickle', 'rb')
model = pickle.load(file)

In [26]:
y_pred = model.predict(X)

In [28]:
pred_df = pd.DataFrame(y_pred, columns = ['Price'])

In [29]:
pred_df.insert(0, 'Name', df['Name'])

In [30]:
pred_df

,Name,Price
0,Mahindra XUV500 W6 2WD,0
1,BMW 5 Series 520d Luxury Line,1
2,Toyota Corolla Altis 1.8 G,0
3,Hyundai i20 Asta 1.4 CRDi,0
4,Mahindra KUV 100 mFALCON G80 K2,0
...,...,...
2515,Maruti Swift VDI,0
2516,Hyundai Xcent 1.1 CRDi S,-1
2517,Mahindra Xylo D4 BSIV,0
2518,Maruti Wagon R VXI,-1


In [31]:
pred_df.to_csv("/content/drive/MyDrive/Temp_hack/test_pred.csv")